In [20]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn import metrics
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

In [2]:
splits = {'train': 'plain_text/train-00000-of-00001.parquet', 'test': 'plain_text/test-00000-of-00001.parquet', 'unsupervised': 'plain_text/unsupervised-00000-of-00001.parquet'}
df = pd.read_parquet("hf://datasets/stanfordnlp/imdb/" + splits["train"])

In [3]:
df.head(10)

,text,label
0,I rented I AM CURIOUS-YELLOW from my video sto...,0
1,"""I Am Curious: Yellow"" is a risible and preten...",0
2,If only to avoid making this type of film in t...,0
3,This film was probably inspired by Godard's Ma...,0
4,"Oh, brother...after hearing about this ridicul...",0
5,I would put this at the top of my list of film...,0
6,Whoever wrote the screenplay for this movie ob...,0
7,"When I first saw a glimpse of this movie, I qu...",0
8,"Who are these ""They""- the actors? the filmmake...",0
9,This is said to be a personal film for Peter B...,0


In [4]:
data = df['text'].tolist()
data[:3]

['I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, e

In [5]:
s = pd.Series(data)
s[:3]

0    I rented I AM CURIOUS-YELLOW from my video sto...
1    "I Am Curious: Yellow" is a risible and preten...
2    If only to avoid making this type of film in t...
dtype: object

In [7]:
stop_words = set(stopwords.words('english'))
def preprocess(text):
    text = text.lower()
    text = ''.join([word for word in text if word not in string.punctuation])
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

In [8]:
s_upd = s.apply(preprocess)
s_upd[:3]

0    rented curiousyellow video store controversy s...
1    curious yellow risible pretentious steaming pi...
2    avoid making type film future film interesting...
dtype: object

In [9]:
clean_df = pd.DataFrame(s_upd, columns=['text'])
clean_df['label'] = df['label']
clean_df.head()

,text,label
0,rented curiousyellow video store controversy s...,0
1,curious yellow risible pretentious steaming pi...,0
2,avoid making type film future film interesting...,0
3,film probably inspired godards masculin fémini...,0
4,oh brotherafter hearing ridiculous film umptee...,0


In [10]:
X = clean_df.text
y = clean_df.label
print(X.shape)
print(y.shape)

(25000,)
(25000,)


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(20000,)
(5000,)
(20000,)
(5000,)


In [12]:
vect = CountVectorizer()

In [13]:
vect.fit(X_train)

In [15]:
X_train_dtm = vect.fit_transform(X_train)
X_train_dtm

<20000x106532 sparse matrix of type '<class 'numpy.int64'>'
	with 2029500 stored elements in Compressed Sparse Row format>

In [16]:
X_test_dtm = vect.transform(X_test)
X_test_dtm

<5000x106532 sparse matrix of type '<class 'numpy.int64'>'
	with 497223 stored elements in Compressed Sparse Row format>

In [17]:
nb = MultinomialNB()

In [18]:
%time nb.fit(X_train_dtm, y_train)

CPU times: total: 0 ns
Wall time: 26.9 ms


MultinomialNB()

In [19]:
y_pred_class = nb.predict(X_test_dtm)

In [21]:
metrics.accuracy_score(y_test, y_pred_class)

0.8602

In [22]:
metrics.confusion_matrix(y_test, y_pred_class)

array([[2189,  326],
       [ 373, 2112]], dtype=int64)

In [23]:
y_pred_prob = nb.predict_proba(X_test_dtm)[:, 1]
y_pred_prob[:10]

array([1.38678219e-05, 1.00000000e+00, 5.79542353e-08, 9.99999994e-01,
       1.00000000e+00, 2.21564439e-07, 7.76638762e-37, 1.00000000e+00,
       8.47919243e-08, 3.95307612e-01])

In [24]:
metrics.roc_auc_score(y_test, y_pred_prob)

0.9263636690920871